In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame
import xarray as xr
import os
import math
from scipy.interpolate import InterpolatedUnivariateSpline
import scipy
from scipy import integrate
import glob

In [ ]:
fold=os.listdir('/directory/')
for fi in fold:
    files=os.listdir('/directory/'+fi)
    
    for j in files:
        
        ds1=xr.open_dataset('/directory/'+fi+'/'+j)
        df_new=ds1.to_dataframe()
        df_new.reset_index(inplace=True)
        
        df_newa = df_new[(df_new['NO_ver'] >0)]
        df_new1 = df_new[(df_new['NO_ver_top'] >0)]
        df_new2=df_newa[(df_newa['tpaltitude'] >= 70) & (df_newa['tpaltitude'] <= 90)]
        df_new3=df_newa[(df_newa['tpaltitude'] >= 115) & (df_newa['tpaltitude'] <= 155)]
        
        df_mx=df_newa[(df_newa['tpaltitude'] >= 110) & (df_newa['tpaltitude'] <= 155)]

        df_new5=df_newa[(df_newa['tpaltitude'] >= 105)]
        df_new4=df_new1[(df_new1['tpaltitude_top'] >= 155) & (df_new1['tpaltitude_top'] <= 250)]
        df_new.dropna(subset=['time','tpaltitude','tplatitude','tplongitude'],inplace=True)
        
        solAP_mean=[]
        solKP_mean=[]
        solDaily_mean=[]
        scSolarZen=[]
        c=[]

        for i in range(1,99):
            solAP_mean.append(df_new[df_new['event']==i].solAP.median())
            solKP_mean.append(df_new[df_new['event']==i].solKP.median())
            scSolarZen.append(df_new[df_new['event']==i].scSolarZen.median())
            solDaily_mean.append(df_new[df_new['event']==i].solf10p7Daily.median())
            c.append(i)
                         
        longitude_mean=[]
        latitude_mean=[]
        time_mean=[] 
        localtime_mean=[]
        date=[]
        ktemp=[]
        nov=[]
        alt1=[]

        for i in range(1,99):
            longitude_mean.append(df_new[df_new['event']==i].tplongitude.median())
            latitude_mean.append(df_new[df_new['event']==i].tplatitude.median())
            time_mean.append(df_new[df_new['event']==i].time.median())
            localtime_mean.append(df_new[df_new['event']==i].tpSolarLT.median())
            date.append(df_new[df_new['event']==i].date.median())
            ktemp.append(df_new5[df_new5['event']==i].ktemp.max())

            nov.append(df_mx[df_mx['event']==i].NO_ver.max())
            alt1.append(df_mx[df_mx.NO_ver == df_mx[df_mx['event']==i].NO_ver.max()].tpaltitude.tolist())
            zz=pd.DataFrame(alt1)
            
            
        z1=[]
        for i in range(1,99):
            a=df_new2[df_new2['event']==i].NO_ver.values
            b=df_new2[df_new2['event']==i].tpaltitude.values
            L=b[::-1]
            N=a[::-1]
            L = [x*100000 for x in L]
            z1.append(scipy.integrate.trapz(N,L))
            
        z2=[] 
        for i in range(1,99):
            a1=df_new3[df_new3['event']==i].NO_ver.values
            b1=df_new3[df_new3['event']==i].tpaltitude.values
            L1=b1[::-1]
            N1=a1[::-1]
            L1 = [y*100000 for y in L1]
            z2.append(scipy.integrate.trapz(N1,L1))
            
        z3=[] 
        for i in range(1,99):
            a3=df_new4[df_new4['event']==i].NO_ver_top.values
            b3=df_new4[df_new4['event']==i].tpaltitude_top.values
            L3=b3[::-1]
            N3=a3[::-1]
            L3 = [y*100000 for y in L3]
            z3.append(scipy.integrate.trapz(N3,L3))   
            
        data = pd.DataFrame({'required parameters'})
        data.insert(14,'max_alt_NO',zz[0])
        
        
        data.to_csv('/directory'+j+'.csv.csv')

In [ ]:
import glob

path = r'/directory/' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)
frame.sort_values(by='date')
fd=frame.dropna()
hr=fd['time_mean']/3600000
fd.insert(4,'HR',hr)

hr1=np.where(hr>=24,fd['HR']-24,fd['HR']-0)
fd.insert(5,'HR1',hr1)
fd.sort_values(by='date')

for i, g in fd.groupby('date'):
    g.to_csv('/directory/{}.csv'.format(i))
